In [40]:
import pandas as pd
import numpy as np
import plotly.express as px
clean = pd.read_csv('data/cleaned_data.csv') 

In [41]:
# group by employeeid and benefit id, take average satisfaction score, first comment, sum usagefrequency, take lastuseddate, take max benefit cost, other columns take the first value
agg_dict = {
    'SatisfactionScore': 'mean',
    'Comments': 'first',
    'UsageFrequency': 'sum',
    'LastUsedDate': 'last',
    'BenefitCost': 'max',
}
# For all other columns, take the first value
other_cols = [col for col in clean.columns if col not in ['EmployeeID','BenefitID'] + list(agg_dict.keys())]
for col in other_cols:
    agg_dict[col] = 'first'

merged = clean.groupby(['EmployeeID','BenefitID'], as_index=False).agg(agg_dict)


## Cost Efficiency and Subcategory Analysis

In [42]:
import warnings
warnings.filterwarnings("ignore")

# Calculate cost-per-usage by BenefitID/BenefitSubType
merged['CostPerUsage'] = merged['BenefitCost'] / merged['UsageFrequency'].replace(0, np.nan)  # Avoid division by zero
merged['CostPerUsage'].fillna(0, inplace=True)  # Fill NaN with 0 for no usage  

# Develop an ROI score (normalize cost-per-usage and satisfaction score)

# Normalize cost-per-usage
max_cost = merged['CostPerUsage'].max()
min_cost = merged['CostPerUsage'].min()
merged['NormalizedCost'] = (merged['CostPerUsage'] - min_cost) / (max_cost - min_cost)

# Normalize satisfaction score
max_satisfaction = merged['SatisfactionScore'].max()
min_satisfaction = merged['SatisfactionScore'].min()
merged['NormalizedSatisfaction'] = (merged['SatisfactionScore'] - min_satisfaction) / (max_satisfaction - min_satisfaction)

# Calculate ROI score
# Those that have no cost or satisfaction will have NaN ROI, which we can handle later
merged['ROI_Score'] = merged['NormalizedSatisfaction'] / merged['NormalizedCost'].replace(0, np.nan)  # Avoid division by zero

# Distribution of cost per usage by BenefitSubType

def create_distribution_plot(df, x_col):
    fig = px.histogram(
        df,
        x=x_col,
        color='BenefitSubType',
        nbins=20,
        category_orders={'BenefitSubType': sorted(df['BenefitSubType'].unique())},
        labels={x_col: x_col},
        title="Distribution of " + x_col + " by Benefit Subtype"
    )
    fig.update_layout(xaxis_title=f'{x_col}', yaxis_title='Count')
    fig.show()

create_distribution_plot(merged, 'CostPerUsage')
create_distribution_plot(merged, 'ROI_Score')

In [43]:
merged.groupby('BenefitSubType').agg({'ROI_Score': 'mean'}).reset_index()

,BenefitSubType,ROI_Score
0,401k Basic Matching,3.309877
1,401k Catch-Up Contributions,4.937111
2,401k High Contribution,11.126834
3,401k Investment Fees,4.182699
4,401k Maximum Matching,3.083640
5,401k Standard Matching,4.525922
6,After-School Care,5.627464
7,Basic Coverage,16.512725
8,Conference Attendance,3.426524
9,Dependent Coverage,17.369603


In [44]:


fig = px.bar(merged.groupby('BenefitSubType').agg({'ROI_Score': 'mean'}).reset_index(),
                x='BenefitSubType',
                y='ROI_Score')
fig.show()

In [45]:
# Identify underutilized high-cost subcategories
# Group by BenefitSubType and calculate mean cost-per-usage and usage frequency, satisfaction score, and rank ROI score
subcategory_stats = merged.groupby('BenefitSubType').agg({
    'CostPerUsage': 'mean',
    'UsageFrequency': 'mean',
    'SatisfactionScore': 'mean',
    'ROI_Score': 'mean'
}).reset_index()

# Define high-cost threshold as the 75th percentile of cost-per-usage
high_cost_threshold = subcategory_stats['CostPerUsage'].quantile(0.75)

# Define low-usage threshold as the 25th percentile of usage frequency
low_usage_threshold = subcategory_stats['UsageFrequency'].quantile(0.25)
roi_usage_treshold = subcategory_stats['ROI_Score'].quantile(0.25)

# Visualise and highlight those above/below thresholds
# Highlight subcategories above the high cost threshold and below the low usage threshold with colored bars

def create_bar_plot_treshold(df, x_col, y_col, title, treshold, treshold_label='max', highlight_above=None):
    # Mapping raw column names to pretty labels
    pretty_labels = {
        'CostPerUsage': 'Cost per Usage',
        'UsageFrequency': 'Usage Frequency',
        'ROI_Score': 'ROI Score'
    }

    # Determine base threshold logic
    if treshold_label == 'max':
        df['ThresholdFlag'] = np.where(df[y_col] > treshold, 'Above Threshold', 'Normal')
    else:
        df['ThresholdFlag'] = np.where(df[y_col] < treshold, 'Below Threshold', 'Normal')

    # Extra highlight for very high values
    if highlight_above is not None:
        df.loc[df[y_col] > highlight_above, 'ThresholdFlag'] = 'Outstanding'

    # Color map with optional extra category
    color_map = {
        'Above Threshold': 'red',
        'Below Threshold': 'red',
        'Normal': 'lightgray',
        'Outstanding': 'darkgoldenrod'
    }

    # Plot
    fig = px.bar(
        df,
        x=x_col,
        y=y_col,
        color='ThresholdFlag',
        color_discrete_map=color_map,
        title=title
    )

    fig.update_yaxes(title_text=pretty_labels.get(y_col, y_col.replace('_', ' ')))
    fig.update_layout(xaxis_tickangle=-90, width=1000, height=500, template='plotly_white')
    fig.add_hline(y=treshold, line_dash="dash", line_color="darkred")

    if highlight_above is not None:
        fig.add_hline(y=highlight_above, line_dash="dash", line_color="darkgoldenrod")

    return fig  # <-- Return figure instead of showing it



create_bar_plot_treshold(merged.groupby('BenefitSubType').agg({'CostPerUsage': 'mean'}).reset_index(), 'BenefitSubType', 'CostPerUsage', 'Average Cost Per Usage by Benefit Subtype', high_cost_threshold, 'max').show()
create_bar_plot_treshold(merged.groupby('BenefitSubType').agg({'UsageFrequency': 'mean'}).reset_index(), 'BenefitSubType', 'UsageFrequency', 'Average Usage Frequency by Benefit Subtype', low_usage_threshold, 'min').show()
create_bar_plot_treshold(merged.groupby('BenefitSubType').agg({'ROI_Score': 'mean'}).reset_index(), 'BenefitSubType', 'ROI_Score', 'Average ROI Score by Benefit Subtype', roi_usage_treshold, 'min',highlight_above=20).show()

# Find those that have high cost and low usage
underutilized_high_cost = subcategory_stats[
    (subcategory_stats['CostPerUsage'] > high_cost_threshold) &
    (subcategory_stats['UsageFrequency'] < low_usage_threshold)
]

print("Underutilized High-Cost Subcategories:")
print(underutilized_high_cost)

# Create quadrant plots (cost vs. usage, cost vs. satisfaction)
fig = px.scatter(subcategory_stats, x='CostPerUsage', y='UsageFrequency',
                 color='BenefitSubType',
                 title='Cost vs Usage Frequency by Benefit Subtype',
                 labels={'CostPerUsage': 'Average Cost per Usage', 'UsageFrequency': 'Average Usage Frequency'},
                 hover_data=['BenefitSubType'])
fig.add_hline(y=low_usage_threshold, line_dash="dash", line_color="red", annotation_text="Low Usage Threshold")
fig.add_vline(x=high_cost_threshold, line_dash="dash", line_color="blue", annotation_text="High Cost Threshold")
fig.show()

# Find those with have high cost and low satisfaction
underutilized_high_cost_satisfaction = subcategory_stats[
    (subcategory_stats['CostPerUsage'] > high_cost_threshold) &
    (subcategory_stats['SatisfactionScore'] < subcategory_stats['SatisfactionScore'].quantile(0.25))
]

print("Underutilized High-Cost Benefits with Low Satisfaction:")
print(underutilized_high_cost_satisfaction[['BenefitSubType', 'CostPerUsage', 'SatisfactionScore']])

# Visualise and highlight those above/below thresholds
fig = px.scatter(subcategory_stats, x='CostPerUsage', y='SatisfactionScore',
                 color='BenefitSubType',
                 title='Cost vs SatisfactionScore by Benefit Subtype',
                 labels={'CostPerUsage': 'Average Cost per Usage', 'SatisfactionScore': 'Average SatisfactionScore'},
                 hover_data=['BenefitSubType'])
fig.add_hline(y=subcategory_stats['SatisfactionScore'].quantile(0.25), line_dash="dash", line_color="red", annotation_text="Low Satisfaction Threshold")
fig.add_vline(x=high_cost_threshold, line_dash="dash", line_color="blue", annotation_text="High Cost Threshold")
fig.show()

Underutilized High-Cost Subcategories:
         BenefitSubType  CostPerUsage  UsageFrequency  SatisfactionScore  \
11      Graduate Degree    195.952842        3.717949           3.115385   
18  On-Site Infant Care    204.423835        3.602510           2.901674   

    ROI_Score  
11   3.015297  
18   2.727476  


Underutilized High-Cost Benefits with Low Satisfaction:
         BenefitSubType  CostPerUsage  SatisfactionScore
0   401k Basic Matching    212.098774           2.878151
15   Individual Courses    188.138088           2.849624
18  On-Site Infant Care    204.423835           2.901674


In [46]:
# Rank benefits by cost efficiency
subcategory_stats['CostEfficiencyRank'] = (subcategory_stats['ROI_Score'].rank(ascending=False)).astype(int)

# Flag which benefits could be removed
# Define conditions for removal - high cost and low usage or low satisfaction
subcategory_stats['RemoveFlag'] = np.where(
    (subcategory_stats['CostPerUsage'] > high_cost_threshold) &
    ((subcategory_stats['UsageFrequency'] < low_usage_threshold) |
    (subcategory_stats['SatisfactionScore'] < subcategory_stats['SatisfactionScore'].quantile(0.25))),
    'Remove', 'Keep'
)
# Find BenefitType
benefit_types = merged[['BenefitType','BenefitSubType']].drop_duplicates().reset_index(drop=True)

# Merge subcategory stats with benefit types
category_stats = subcategory_stats.merge(
    benefit_types,
    on='BenefitSubType',
    how='left'
)

# Display categories that could be removed
removable_categories = category_stats[category_stats['RemoveFlag'] == 'Remove']
print("Removable Categories:")
print(removable_categories[['BenefitType','BenefitSubType', 'CostPerUsage', 'UsageFrequency', 'SatisfactionScore', 'ROI_Score','RemoveFlag']])


Removable Categories:
              BenefitType       BenefitSubType  CostPerUsage  UsageFrequency  \
0         Retirement Plan  401k Basic Matching    212.098774        4.689076   
11  Tuition Reimbursement      Graduate Degree    195.952842        3.717949   
15  Tuition Reimbursement   Individual Courses    188.138088        4.281955   
18              Childcare  On-Site Infant Care    204.423835        3.602510   

    SatisfactionScore  ROI_Score RemoveFlag  
0            2.878151   3.309877     Remove  
11           3.115385   3.015297     Remove  
15           2.849624   3.558484     Remove  
18           2.901674   2.727476     Remove  


In [47]:
category_stats

,BenefitSubType,CostPerUsage,UsageFrequency,SatisfactionScore,ROI_Score,CostEfficiencyRank,RemoveFlag,BenefitType
0,401k Basic Matching,212.098774,4.689076,2.878151,3.309877,25,Remove,Retirement Plan
1,401k Catch-Up Contributions,121.324838,4.139442,2.790837,4.937111,16,Keep,Retirement Plan
2,401k High Contribution,59.199464,5.003759,2.918546,11.126834,8,Keep,Retirement Plan
3,401k Investment Fees,151.009728,4.621951,3.085366,4.182699,20,Keep,Retirement Plan
4,401k Maximum Matching,192.073667,4.566524,2.962089,3.083640,26,Keep,Retirement Plan
5,401k Standard Matching,136.411769,3.893617,3.005319,4.525922,17,Keep,Retirement Plan
6,After-School Care,109.681167,3.628692,3.099156,5.627464,13,Keep,Childcare
7,Basic Coverage,37.809574,4.522449,2.963265,16.512725,7,Keep,Life Insurance
8,Conference Attendance,199.862171,4.333333,3.064919,3.426524,24,Keep,Professional Development
9,Dependent Coverage,40.842833,4.544000,3.072000,17.369603,6,Keep,Life Insurance


In [48]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_metrics_for_benefit_type(df, benefit_type):
    metrics = {
        'ROI_Score': 'ROI Score',
        'CostPerUsage': 'Cost per Usage',
        'UsageFrequency': 'Usage Frequency',
        'SatisfactionScore': 'Satisfaction Score'
    }
    
    # Filter to the chosen BenefitType
    subset = df[df['BenefitType'] == benefit_type]
    
    # Group by subtype
    grouped = subset.groupby('BenefitSubType').agg({
        'ROI_Score': 'mean',
        'CostPerUsage': 'mean',
        'UsageFrequency': 'mean',
        'SatisfactionScore': 'mean'
    }).reset_index()
    
    # Create subplot layout (2x2)
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[f'{label} by Benefit Subtype' for label in metrics.values()]
    )
    
    # Add each metric as a subplot
    row_col_positions = [(1,1), (1,2), (2,1), (2,2)]
    for (col, label), (row, col_pos) in zip(metrics.items(), row_col_positions):
        fig.add_trace(
            go.Bar(
                x=grouped['BenefitSubType'],
                y=grouped[col],
                name=label
            ),
            row=row, col=col_pos
        )
    
    # Update layout
    fig.update_layout(
        height=1000,
        width=1200,
        showlegend=False,
        template='plotly_white',
        title_text=f'Metrics by Benefit Subtype ({benefit_type})'
    )
    
    # Rotate x-axis labels
    fig.update_xaxes(tickangle=-45)
    
    fig.show()


In [49]:

# Example of possible to remove subbenfits shown with metrices:
plot_metrics_for_benefit_type(category_stats, 'Tuition Reimbursement')

plot_metrics_for_benefit_type(category_stats, 'Retirement Plan')

plot_metrics_for_benefit_type(category_stats, 'Childcare')

plot_metrics_for_benefit_type(category_stats, 'Gym Membership')

plot_metrics_for_benefit_type(category_stats, 'Health Insurance')


## Findings 

- We looked for subcategories and their individual cost, how satisfied employees were on average, how much frequently used they were on average.
- Subcategories that were very costly and were not used a lot are - On-Site Infant Care and Graduate Degree. These subcategories seem to be  underused and costly.
- Subcategories that were very costly (within 25% most costly categories) and on average employees were not satisfied (25% least satisfied) with - 401k Basic Matching,  Individual Courses, On-Site Infant Care. These subcategories seem to be unsatisfactory and costly.
- Conditions for removal were chosen as subcategories that are both costly and underused or unsatisfactory.
- There were all together 3 BenefitTypes and 4 Subcategories that had 'Remove' in the flag column, and should be consider for reviewing or discontinuing it. These were: 
    - Retirement Plan, i.e. 401k Basic Matching (frequently used, but least satisfactory on average), there are different retirement benefits with higher ROI 
    - Tuition Reimbursement, i.e.Graduate Degree and Individual Courses, there are other subbenefits in the category that could be a better alternative such as Professional Certification which gives the highest Return on Investement
    - Chilcare, i.e On-Site Infant Care that is not frequent and not satisfactory in general, suggesting to find better substitutes
- Within the Benefit Types, there are benefitsubtypes that are too expensive for a company and can be "substituted" for a better alternative. If we look at for example Tuition Reimbursment: Graduate Degrees have very high ROI because they are costly, however when it comes to satisfaction, they are very similar to Professional Courses whicch ROI is much higher due to lower cost. In general, they provide the same benefit - knowledge, gain in expertise.
